# Bike sharing in Washington
<em>Individual Project Python<em>
___

                              Master in Business Analytics & Big data
                                 Professor Juan Luis Cano Rodriguez
                                           Martin Hofbauer
                                              _________

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-Loading" data-toc-modified-id="Data-Loading-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Loading</a></span></li><li><span><a href="#Preparation-for-Baseline" data-toc-modified-id="Preparation-for-Baseline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preparation for Baseline</a></span><ul class="toc-item"><li><span><a href="#Renaming-the-Colums" data-toc-modified-id="Renaming-the-Colums-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Renaming the Colums</a></span></li><li><span><a href="#Applying-the-Corrrect-Datatypes,-Normalizing-and-DummyEncoding" data-toc-modified-id="Applying-the-Corrrect-Datatypes,-Normalizing-and-DummyEncoding-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Applying the Corrrect Datatypes, Normalizing and DummyEncoding</a></span></li><li><span><a href="#Splitting-the-Dataset" data-toc-modified-id="Splitting-the-Dataset-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Splitting the Dataset</a></span></li></ul></li><li><span><a href="#Baseline-Predictions" data-toc-modified-id="Baseline-Predictions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Baseline Predictions</a></span><ul class="toc-item"><li><span><a href="#Linear-Model-Function" data-toc-modified-id="Linear-Model-Function-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Linear Model Function</a></span></li><li><span><a href="#Predicting-Count,-Casual-and-Registered-Seperately" data-toc-modified-id="Predicting-Count,-Casual-and-Registered-Seperately-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Predicting Count, Casual and Registered Seperately</a></span></li><li><span><a href="#Summing-Casual-and-Registered-Predictions" data-toc-modified-id="Summing-Casual-and-Registered-Predictions-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Summing Casual and Registered Predictions</a></span></li></ul></li></ul></div>

## Libraries

<a id="lib"></a>

In [1]:
from distributed import Client, progress
import dask
import dask.dataframe as dd
import dask.array as da
import numpy as np
import dask_ml
from dask_ml.preprocessing import Categorizer, DummyEncoder
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from dask_ml.preprocessing import StandardScaler
from dask_ml.linear_model import LinearRegression
from dask_ml.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
%load_ext nb_black

C:\Users\Martin\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\Martin\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


<IPython.core.display.Javascript object>

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:56036 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.10 GB


<IPython.core.display.Javascript object>

## Data Loading

In [3]:
hour_data = dd.read_csv(
    "https://gist.githubusercontent.com/geraldwal/b5a83f4c670abe0a662abce558e5d433/raw/bce4bbfc63355606e4503964e25798b5d2190b9b/hour%2520-%2520Python%2520Bike%2520Sharing",
    sep=",",
    parse_dates=["dteday"],
)

<IPython.core.display.Javascript object>

## Preparation for Baseline

### Renaming the Colums

In [4]:
hour_data

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
npartitions=1,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


<IPython.core.display.Javascript object>

In [5]:
hour_data = hour_data.rename(
    columns={
        "weathersit": "weather",
        "mnth": "month",
        "hr": "hour",
        "hum": "humidity",
        "cnt": "count",
        "yr": "year",
    }
)

<IPython.core.display.Javascript object>

In [6]:
# new_columns = [
#     "instant",
#     "dteday",
#     "season",
#     "year",
#     "month",
#     "hour",
#     "holiday",
#     "weekday",
#     "workingday",
#     "weather",
#     "temp",
#     "atemp",
#     "humidity",
#     "windspeed",
#     "casual",
#     "registered",
#     "count",
# ]
# hour_data= hour_data.rename(columns=dict(zip(hour_data.columns, new_columns)))

<IPython.core.display.Javascript object>

In [7]:
hour_data = hour_data.drop(["dteday"], axis=1)

<IPython.core.display.Javascript object>

In [8]:
hour_data_cl = hour_data.copy()

<IPython.core.display.Javascript object>

### Applying the Corrrect Datatypes, Normalizing and DummyEncoding

In [9]:
ss = StandardScaler()
norm = ss.fit_transform(
    hour_data.drop(
        [
            "season",
            "year",
            "month",
            "hour",
            "holiday",
            "weekday",
            "workingday",
            "weather",
            "casual",
            "registered",
            "count",
        ],
        axis=1,
    )
)

<IPython.core.display.Javascript object>

In [10]:
cat = hour_data.loc[
    :,
    [
        "season",
        "year",
        "month",
        "hour",
        "holiday",
        "weekday",
        "workingday",
        "weather",
        "casual",
        "registered",
        "count",
    ],
]
hour_data = dd.concat([cat, norm], axis=1)

C:\Users\Martin\Anaconda3\lib\site-packages\dask\dataframe\multi.py:590: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn("Concatenating dataframes with unknown divisions.\n"


<IPython.core.display.Javascript object>

In [11]:
categs = [
    "season",
    "year",
    "month",
    "hour",
    "holiday",
    "weekday",
    "workingday",
    "weather",
]

<IPython.core.display.Javascript object>

In [12]:
pipeline = make_pipeline(Categorizer(columns=categs), DummyEncoder(columns=categs))
hour_data_cl_hot_encoded = pipeline.fit_transform(hour_data)

<IPython.core.display.Javascript object>

In [13]:
hour_data_cl_hot_encoded = hour_data_cl_hot_encoded.rename(
    columns={
        "holiday_0": "no_holiday",
        "holiday_1": "yes_holiday",
        "month_1": "jan",
        "month_2": "feb",
        "month_3": "mar",
        "month_4": "apr",
        "month_5": "may",
        "month_6": "jun",
        "month_7": "jul",
        "month_8": "aug",
        "month_9": "sep",
        "month_10": "oct",
        "month_11": "nov",
        "month_12": "dec",
        "weekday_0": "sun",
        "weekday_1": "mon",
        "weekday_2": "tue",
        "weekday_3": "wed",
        "weekday_4": "thu",
        "weekday_5": "fri",
        "weekday_6": "sat",
        "season_1": "winter",
        "season_2": "spring",
        "season_3": "summer",
        "season_4": "autumn",
    }
)

<IPython.core.display.Javascript object>

### Splitting the Dataset

The intital Split function needed to be adapted a bit, including a .loc after the X and y. Furthermore, the trainsize needed to be reduced by 1 in order not to have rows included in both train and test.

In [14]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X.loc[0 : train_size - 1],
        X.loc[train_size : len(dataset)],
        y.loc[0 : train_size - 1],
        y.loc[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

<IPython.core.display.Javascript object>

In [15]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = split_data(
    hour_data_cl_hot_encoded, "registered"
)
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(
    hour_data_cl_hot_encoded, "casual"
)
x_train_count, x_test_count, y_train_count, y_test_count = split_data(
    hour_data_cl_hot_encoded, "count"
)

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


<IPython.core.display.Javascript object>

In [16]:
x_train_reg_drop, x_test_reg_drop = (
    x_train_reg.drop(["casual", "count"], axis=1),
    x_test_reg.drop(["casual", "count"], axis=1),
)

<IPython.core.display.Javascript object>

In [17]:
x_train_reg_arr, x_test_reg_arr, y_train_reg_arr, y_test_reg_arr = (
    x_train_reg_drop.values,
    x_test_reg_drop.values,
    y_train_reg.values,
    y_test_reg.values,
)

<IPython.core.display.Javascript object>

In [18]:
x_train_casual_drop, x_test_casual_drop = (
    x_train_casual.drop(["registered", "count"], axis=1),
    x_test_casual.drop(["registered", "count"], axis=1),
)

<IPython.core.display.Javascript object>

In [19]:
x_train_casual_arr, x_test_casual_arr, y_train_casual_arr, y_test_casual_arr = (
    x_train_casual_drop.values,
    x_test_casual_drop.values,
    y_train_casual.values,
    y_test_casual.values,
)

<IPython.core.display.Javascript object>

In [20]:
x_train_count_drop, x_test_count_drop = (
    x_train_count.drop(["casual", "registered"], axis=1),
    x_test_count.drop(["casual", "registered"], axis=1),
)

<IPython.core.display.Javascript object>

In [21]:
x_train_count_arr, x_test_count_arr, y_train_count_arr, y_test_count_arr = (
    x_train_count_drop.values,
    x_test_count_drop.values,
    y_train_count.values,
    y_test_count.values,
)

<IPython.core.display.Javascript object>

## Baseline Predictions

### Linear Model Function

In [22]:
def score_lin(X_train, X_test, y_train, y_test):
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Intercept:", lm.intercept_)
    print("Coefficients:", lm.coef_)
    print("Mean squared error (MSE): {:.2f}".format(mean_squared_error(y_test, y_pred)))
    print(
        "Variance score (R2): {:.2f}".format(
            r2_score(y_test.compute(), y_pred.compute())
        )
    )
    return y_pred

<IPython.core.display.Javascript object>

### Predicting Count, Casual and Registered Seperately

In [23]:
##  baseline predictions and score for registered
# train_dropped_target = x_train_reg.drop(["casual", "count"], axis = 1)
# test_dropped_target = x_test_reg.drop(["casual", "count"], axis = 1)
baseline_registered_pred = score_lin(
    x_train_reg_arr, x_test_reg_arr, y_train_reg_arr, y_test_reg_arr
)

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Intercept: -44.74361284450108
Coefficients: [ 1.10969963e+02  2.92534179e+01  7.88507302e+00  1.45519202e+01
 -1.06502491e+01 -1.44722552e+00 -1.41629636e+01  5.56213897e+00
 -2.34379178e-01  2.53669788e+01 -1.04739472e+01  1.43495793e+01
  1.31171448e+00  3.77377722e+00  6.30218198e+00  3.84271827e+00
  1.64515077e+01  1.08503312e+01 -4.89201674e+00  6.00568681e+00
  1.80724429e+01 -7.86658830e+00 -2.02444591e+01 -1.91199493e+01
 -9.49405748e+01 -1.09363673e+02 -1.16434725e+02 -1.25250483e+02
 -1.27962050e+02 -1.13712780e+02 -5.99804051e+01  6.32074192e+01
  1.87091775e+02  4.10688965e+01 -2.22798122e+01 -7.69162155e+00
  2.33513589e+01  1.71206440e+01 -5.84744488e-03  8.53106244e+00
  6.89993983e+01  2.19016569e+02  2.01780533e+02  1.07723541e+02
  4.02080955e+01 -1.28020888e+00 -3.35645926e+01 -6.72584818e+01
  1.52036979e+01 -3.28202987e+00 -2.74811988e+00 -1.09721290e+01
  1.57074390e+00  7.56988058e+00  9.02710714e+00  9.50212751e+00
  5.45329847e+00 -1.00020730e+01  1.41343728e+

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


Mean squared error (MSE): 11519.35


C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Variance score (R2): 0.63


<IPython.core.display.Javascript object>

In [24]:
##  baseline predictions and score for casual
# train_dropped_target = x_train_casual.drop(["registered", "count"], axis = 1)
# test_dropped_target = x_test_casual.drop(["registered", "count"], axis = 1)
baseline_casual_pred = score_lin(
    x_train_casual_arr, x_test_casual_arr, y_train_casual_arr, y_test_casual_arr
)

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Intercept: 0.7003294967016319
Coefficients: [ 12.09524023  -6.92504845  14.78545681   1.42186829  -5.29528659
  -2.23612591  -1.1198612    8.09247877   0.38958631   0.83759887
 -10.39176206  14.26413313  -6.33708275  -6.45776987   4.68995497
   3.79204831   5.63442167  -1.15984132  -1.25546851   4.13981814
  13.25055119  16.8926303   11.7338343    8.14277593 -16.96309822
 -19.9244694  -21.00147895 -23.22366705 -23.73005142 -21.83529832
 -18.47730516 -12.02978332  -3.55533918   2.58740343  15.08216326
  24.60616985  30.53709392  32.75746749  34.71562433  33.85806263
  33.43115754  36.62377151  26.51221575  16.67452245   5.29172039
  -1.723133    -6.48690446 -12.43288534   5.83861014   6.07302668
  14.22621113  10.19388071  -0.06427551  -3.18396696  -3.09746635
  -3.25115511   4.1085642   12.92784504  -8.79902227   5.35033327
   2.07158614  -5.62694479]


C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


Mean squared error (MSE): 1075.61


C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Variance score (R2): 0.54


<IPython.core.display.Javascript object>

In [25]:
##  baseline predictions and score for count
# train_dropped_target = x_train_count_arr.drop(["casual", "registered"], axis = 1)
# test_dropped_target = x_test_count_arr.drop(["casual", "registered"], axis = 1)
baseline_registered_count = score_lin(
    x_train_count_arr, x_test_count_arr, y_train_count_arr, y_test_count_arr
)

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Intercept: -46.42112739970177
Coefficients: [ 148.66141376   22.60485296   22.67706131   15.96465478  -15.94294589
   -3.68236923  -17.52717982   11.43352745   -2.06408918   23.97717062
  -22.56348451   26.43423057   -8.27196826   -5.96714879    7.66168074
    4.24776438   18.66064902    6.22753287   -9.64980102    6.60016121
   27.73752731    5.4050132   -12.17358001  -14.67380203 -116.70899821
 -134.09366864 -142.24206468 -153.28042062 -156.49856593 -140.35480875
  -83.26472838   46.37095242  178.73004417   38.85054416  -12.00259646
   12.11060736   49.08520415   45.07535983   29.90706797   37.58642202
   97.62746904  250.83693165  223.4889404   119.59354425   40.69506859
   -7.80853412  -44.85723372  -84.49724908   15.07434172   -3.16770352
    8.74035944   -3.51544206   -1.24139371    1.63980441    3.18344623
    3.50487695    6.81555993    0.85460769    3.27248246   15.60130956
    6.11664937  -45.33401064]


C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices


Mean squared error (MSE): 14814.57


C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


Variance score (R2): 0.63


<IPython.core.display.Javascript object>

### Summing Casual and Registered Predictions

In [26]:
pred_summed = baseline_registered_pred + baseline_casual_pred

<IPython.core.display.Javascript object>

In [27]:
def r2score(x, y):
    s = explained_variance_score(x, y)
    return s

<IPython.core.display.Javascript object>

In [28]:
r2score(x_test_casual["count"].compute(), pred_summed.compute())

C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  contains = index in indices
C:\Users\Martin\Anaconda3\lib\site-packages\dask\blockwise.py:595: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sub[blockwise_token(i)] = blockwise_token(indices.index(index))


0.6377927222843567

<IPython.core.display.Javascript object>